In [92]:
%pip install awswrangler
%pip install mlflow
%pip install optuna

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [93]:
import awswrangler as wr

import mlflow

# Para que funciones, todos nuestros scripts debemos exportar las siguientes variables de entorno
%env AWS_ACCESS_KEY_ID=minio   
%env AWS_SECRET_ACCESS_KEY=minio123 
%env MLFLOW_S3_ENDPOINT_URL=http://localhost:9000
%env AWS_ENDPOINT_URL_S3=http://localhost:9000
#%env MLFLOW_S3_ENDPOINT_URL=http://192.168.0.21:9000
#%env AWS_ENDPOINT_URL_S3=http://192.168.0.21:9000

env: AWS_ACCESS_KEY_ID=minio
env: AWS_SECRET_ACCESS_KEY=minio123
env: MLFLOW_S3_ENDPOINT_URL=http://localhost:9000
env: AWS_ENDPOINT_URL_S3=http://localhost:9000


# Búsqueda de mejor modelo e hiperparámetros

Dado nuestro dataset de Heart Disease, el cual ya pasó por el proceso de ETL y se encuentra en nuestro S3 bucket, vamos a realizar una búsqueda de cual seria el mejor modelo y que hiperparametros usar.

La búsqueda de hiperparametros la haremos usando Optuna y el tracking será realizado mediante MLFlow.

OBS: Para la confección de esta notebook, nos basamos en el tutorial de [MLFlow](https://mlflow.org/docs/latest/traditional-ml/hyperparameter-tuning-with-child-runs/notebooks/index.html).

In [94]:
mlflow_server = "http://localhost:5001"
#mlflow_server = "http://192.168.0.21:5001"

mlflow.set_tracking_uri(mlflow_server)

In [95]:
# Cargamos los datos para realizar nuestro estudio.
# OBS, no vamos a cargar los datos de testing, nada de Data leakage por aquí
X_train =  wr.s3.read_csv("s3://data/final/train/heart_X_train.csv")
y_train =  wr.s3.read_csv("s3://data/final/train/heart_y_train.csv")

X_test =  wr.s3.read_csv("s3://data/final/test/heart_X_test.csv")
y_test =  wr.s3.read_csv("s3://data/final/test/heart_y_test.csv")

## Investigamos la correlación de features con la variable objetivo

Antes de profundizar en el proceso de construcción de modelo, es esencial comprender las relaciones entre nuestras features  y la variable objetivo. Por lo que vamos a realizar un gráfico que indica el coeficiente de correlación de cada feature en relación con la variable objetivo. Esto nos sirve para:

- Evitar data leakage: Debemos asegurarnos de que ninguna característica se correlacione perfectamente con el objetivo (un coeficiente de correlación de aproximadamente 1.0). Si existe tal correlación, es una señal de que nuestro conjunto de datos podría estar "filtrando" información sobre el objetivo. 

- Garantizar relaciones significativas: Idealmente, nuestras características deberían tener algún grado de correlación con el objetivo. Inclusive si estamos trabajando con un problema de clasificación, aunque los resultados no son tan importantes como en un caso de regresión.

- Auditoría y trazabilidad: Loggear esta visualización de correlación con nuestra ejecución principal de MLflow garantiza la trazabilidad. Proporciona una instantánea de las características de los datos en el momento del entrenamiento del modelo, lo cual es invaluable para propósitos de auditoría y replicabilidad.

In [96]:
from plots import plot_correlation_with_target, plot_information_gain_with_target

In [97]:
# Dado que estamos usando como tracking a MLFlow, mostrar los gráficos aquí no tiene sentido.
correlation_plot = plot_correlation_with_target(X_train, y_train)
information_gain_plot = plot_information_gain_with_target(X_train, y_train)

## Arrancamos a experimentar

In [98]:
import datetime
import optuna

from mlflow.models import infer_signature
from mlflow_aux import get_or_create_experiment

from optuna_aux import champion_callback, objective

from sklearn.svm import SVC 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

# Optuna es un poco verboso, dejamos que solo nos muestre logs de errores
optuna.logging.set_verbosity(optuna.logging.ERROR)

Antes de poder realizar experimentos, vamos a crear el experimento en MLFLow, pero para evitar desorden, vamos a usar una función que se fije primero si el experimento existe, si esto es así, devuelve su ID.

Además creamos el nombre del run padre con el que vamos a ir registrando las ejecuciones.

In [99]:
# Creemos el experimento
experiment_id = get_or_create_experiment("Heart Disease")
print(experiment_id)

run_name_parent = "best_hyperparam_"  + datetime.datetime.today().strftime('%Y/%m/%d-%H:%M:%S"')

1


Ya con todo seteado, vamos a ejecutar la optimización usando Optuna, el cual realiza una búsqueda Bayesiana, la cual es más eficiente que una búsqueda de grilla tradicional. La desventaja es que es más difícil de paralelizar.

In [100]:
import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature

artifact_path = "model"

with mlflow.start_run(experiment_id=experiment_id, run_name=run_name_parent, nested=True) as run:
    # Optuna
    study = optuna.create_study(direction="maximize")
    study.optimize(lambda trial: objective(trial, X_train, y_train, experiment_id),
                   n_trials=250, callbacks=[champion_callback])

    # Métricas/params del mejor
    mlflow.log_params(study.best_params)
    mlflow.log_metric("best_train_f1", study.best_value)
    mlflow.set_tags({
        "project": "Heart Disease",
        "optimizer_engine": "optuna",
        "model_family": "sklearn",
        "feature_set_version": 1,
    })

    # Entrena el mejor modelo
    if study.best_params["classifier"] == "SVC_linear":
        model = SVC(C=study.best_params["svc_c"], kernel="linear", gamma="scale")
    elif study.best_params["classifier"] == "SVC_poly":
        model = SVC(C=study.best_params["svc_c"], kernel="poly",
                    gamma="scale", degree=study.best_params["svc_poly_degree"])
    elif study.best_params["classifier"] == "SVC_rbf":
        model = SVC(C=study.best_params["svc_c"], kernel="rbf", gamma="scale")
    elif study.best_params["classifier"] == "DecisionTreeClassifier":
        model = DecisionTreeClassifier(max_depth=study.best_params["tree_max_depth"])
    else:
        model = RandomForestClassifier(max_depth=study.best_params["rf_max_depth"],
                                       n_estimators=study.best_params["rf_n_estimators"])

    model = model.fit(X_train, y_train.to_numpy().ravel())

    # Test y métrica
    y_pred = model.predict(X_test)
    f1_score_val = f1_score(y_test.to_numpy().ravel(), y_pred)
    mlflow.log_metric("test_f1", f1_score_val)

    # Otros artefactos
    mlflow.log_figure(figure=correlation_plot, artifact_file="correlation_plot.png")
    mlflow.log_figure(figure=information_gain_plot, artifact_file="information_gain_plot.png")

    # Firma y log del modelo en el run (y registro en DEV si querés)
    signature = infer_signature(X_train, model.predict(X_train))
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path=artifact_path,
        signature=signature,
        serialization_format="cloudpickle",
        registered_model_name="heart_disease_model_dev",
        metadata={"model_data_version": 1},
    )

    run_id = run.info.run_id
    model_uri = f"runs:/{run_id}/{artifact_path}"


🏃 View run Trial: 0 at: http://localhost:5001/#/experiments/1/runs/67f97076e59e4f2eac91d40c856a89ea
🧪 View experiment at: http://localhost:5001/#/experiments/1
Initial trial 0 achieved value: 0.7372139808981915
🏃 View run Trial: 1 at: http://localhost:5001/#/experiments/1/runs/f581ccdf9e6044ca9a5c6acb93249e6f
🧪 View experiment at: http://localhost:5001/#/experiments/1
🏃 View run Trial: 2 at: http://localhost:5001/#/experiments/1/runs/24f15559590a4131b0f5589a337c68fb
🧪 View experiment at: http://localhost:5001/#/experiments/1
Trial 2 achieved value: 0.7978335478335479 with  7.5980% improvement
🏃 View run Trial: 3 at: http://localhost:5001/#/experiments/1/runs/cdf8c36b56d14ea8988e910040c8329a
🧪 View experiment at: http://localhost:5001/#/experiments/1
🏃 View run Trial: 4 at: http://localhost:5001/#/experiments/1/runs/bbfe1773e13047809ce4f8ed211cf613
🧪 View experiment at: http://localhost:5001/#/experiments/1
🏃 View run Trial: 5 at: http://localhost:5001/#/experiments/1/runs/0761dc0c20fc4

2025/09/25 20:31:33 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
Successfully registered model 'heart_disease_model_dev'.
2025/09/25 20:31:38 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: heart_disease_model_dev, version 1
Created version '1' of model 'heart_disease_model_dev'.


🏃 View run best_hyperparam_2025/09/25-20:28:59" at: http://localhost:5001/#/experiments/1/runs/d487f28e1037429aa1dc10ccf62ee295
🧪 View experiment at: http://localhost:5001/#/experiments/1


## Testeando el modelo

Una vez que el modelo fue entrenado, podemos levantarlo y testearlo de una forma agnóstica a donde está guardado.

In [101]:
loaded = mlflow.sklearn.load_model(model_uri)

In [102]:
import numpy as np
test_data = [-1.0431146438366603, 0.6689936080056726, 0.5955141571109206, 1.4218278518511829, -0.3955054753168235,
             -0.195684619877533, 1.445528359737701, -0.8782783888787548, -0.4354941703556927, -0.6313862911472252,
             1.0752906583803283, -0.0987729596649589, 0.957427107756338, 1.1071614388213236, -0.2991215208080594,
             -0.5494422557947561, -0.362142984170074, 4.690415759823429, -0.2253029545296664, 1.1980376111153852]
loaded.predict(np.array(test_data).reshape([1, -1]))

c:\Users\Jose\anaconda3\envs\ceia-iia-env-01\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


array([0])

## Registramos el modelo 

Realizamos el registro del modelo en MLflow. En este registro se pone el modelo productivo que luego se usará para servir en formato on-line.

In [103]:
import mlflow
from mlflow import MlflowClient
from mlflow.exceptions import RestException

client = MlflowClient()
name = "heart_disease_model_prod"
desc = "This classifier detects if a person has a heart disease"

try:
    client.create_registered_model(name=name, description=desc)
except RestException as e:
    # Si ya existe, ignoramos el error RESOURCE_ALREADY_EXISTS
    if getattr(e, "error_code", "") != "RESOURCE_ALREADY_EXISTS":
        raise

version_tags = {}
version_tags.update(model.get_params())
version_tags["model"] = type(model).__name__
version_tags["f1-score"] = f1_score_val

new_mv = mlflow.register_model(model_uri=model_uri, name=name)

for k, v in version_tags.items():
    client.set_model_version_tag(name=name, version=new_mv.version, key=str(k), value=str(v))

client.set_registered_model_alias(name=name, alias="champion", version=new_mv.version)


Registered model 'heart_disease_model_prod' already exists. Creating a new version of this model...
2025/09/25 20:31:38 WARNING mlflow.tracking._model_registry.fluent: Run with id d487f28e1037429aa1dc10ccf62ee295 has no artifacts at artifact path 'model', registering model based on models:/m-ff82f401a1a845808edb3fec1c67c671 instead
2025/09/25 20:31:39 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: heart_disease_model_prod, version 1
Created version '1' of model 'heart_disease_model_prod'.
